# GSOC 2024
## Jonas Koziorek

* **Organization**: The Julia Language (https://julialang.org/)
* **Mentor**: George Datseris (https://github.com/Datseris)

# Project Goals

The primary objective of this project was to develop a unified interface for periodic orbit detection algorithms. These algorithms are essential tools in the study of dynamical systems, particularly for identifying recurring trajectories in systems that exhibit complex or chaotic behavior. While several algorithms already existed in Julia packages such as DynamicalSystems.jl and BifurcationKit.jl, they lacked a cohesive structure and synchronization. My task involved not only unifying these algorithms under a common interface but also implementing new ones based on recent advancements in the field.
Additionally, the project aimed to bridge the gap between DynamicalSystems.jl and BifurcationKit.jl, offering users a more unified experience.

# My contribution

In the beginning we decided that all the new code should go into a new Julia package [**PeriodicOrbits.jl**](https://github.com/JuliaDynamics/PeriodicOrbits.jl) which falls under the [**JuliaDynamics** ecosystem](https://github.com/JuliaDynamics). In the first half of the program I devised a public a public interface to which all the detection algorithms should follow. In the second half I focused on developing new detection algorithms and writing documentation.

## Public Interface

The public interface encompasses these features:

1. Custom type for periodic orbits
2. Custom type for initial guesses which are required in the detection algorithms
3. Abstract supertype for all the detection algorithms
4. Function which computes minimal period of the periodic orbit
5. Function that evaluates local stability properties of the periodic orbit
6. Function that computes distance between two periodic orbits
7. Function that finds unique periodic orbits in a list of periodic orbits

All the features work for both discrete-time and continuous-time dynamical systems.

## Implemented Algorithms

For discrete-time dynamical systems there are two options - Schmelcher & Diakonos algorithm or Davidchack & Lai algorithm.
For continuous-time dynamical systems there are two options - Optimized Shooting algorithm or discrete-time algorithms in combination with Poincaré map.

## My Pull Requests

1. [PR #2](https://github.com/JuliaDynamics/PeriodicOrbits.jl/pull/2)
2. [PR #3](https://github.com/JuliaDynamics/PeriodicOrbits.jl/pull/3)
3. [PR #5](https://github.com/JuliaDynamics/PeriodicOrbits.jl/pull/5)
4. [PR #6](https://github.com/JuliaDynamics/PeriodicOrbits.jl/pull/6)
5. [PR #7](https://github.com/JuliaDynamics/PeriodicOrbits.jl/pull/7)
6. [PR #15](https://github.com/JuliaDynamics/PeriodicOrbits.jl/pull/15)
7. [PR #211](https://github.com/JuliaDynamics/DynamicalSystemsBase.jl/pull/211)


# Current State

The public interface is finished together. Two previous detection algorithms that were in ChaosTools.jl were ported to PeriodicOrbits.jl. A new detection algorithm aimed for continuous-time dynamical system was added. All of the current work is backed with tests and documentation.

# What is left to do

**Integration with BifurcationKit.jl**: Strengthening the connection between PeriodicOrbits.jl and other libraries such as BifurcationKit.jl would enhance the flexibility and applicability of the package.

**Further Algorithm Development**: While the current selection is functional, there is potential to incorporate more sophisticated methods. For instance, I started implementing the [**Fictitious time integration method**](https://www.sciencedirect.com/science/article/pii/S0020746223000409) but could not complete it within the program's timeframe. Continuing this work would broaden the range of tools available to users.


# Challenges

Two algorithms I initially explored turned out to be unsuitable:

1. [**Numerical detection of unstable periodic orbits in continuous-time dynamical systems with chaotic behaviors**](https://npg.copernicus.org/articles/14/615/2007/) - I implemented this algorithm but I couldn't make it work correctly. After some time I dropped my efforts.
2. [**Determination of Unstable Limit Cycles in Chaotic Systems by the Method of Unrestricted Harmonic Balance**](https://www.degruyter.com/document/doi/10.1515/zna-1991-0605/html) - This method required extensive manual intervention, making it impractical for automation in a general-use package.

# What I learned

Throughout this project, I gained substantial experience in working with GitHub and the open-source development process, particularly how to structure contributions, manage pull requests, and use CI for automated testing and documentation deployment. I deepened my understanding of numerical methods in dynamical systems and became more familiar with the JuliaDynamics ecosystem. Working through scientific papers and translating algorithms into practical code sharpened both my research and implementation skills.

# Code Showcase

Let's attempt to detect a periodic orbit of the Lorenz system. First, we define the Lorenz system itself.

In [ ]:
import Pkg
Pkg.activate("/home/jonas/gsoc/PeriodicOrbits.jl") # here comes location of local PeriodicOrbits.jl repository

using PeriodicOrbits

function lorenz(u0=[0.0, 10.0, 0.0]; σ = 10.0, ρ = 28.0, β = 8/3)
    return CoupledODEs(lorenz_rule, u0, [σ, ρ, β])
end

@inbounds function lorenz_rule(u, p, t)
    du1 = p[1] * (u[2] - u[1])
    du2 = u[1] * (p[2] - u[3]) - u[2]
    du3 = u[1] * u[2] - p[3] * u[3]
    return SVector{3}(du1, du2, du3)
end

ds = lorenz()

  Activating project at `~/gsoc/PeriodicOrbits.jl`
┌ Warning: The project dependencies or compat requirements have changed since the manifest was last resolved.
│ It is recommended to `Pkg.resolve()` or consider `Pkg.update()` if necessary.
└ @ Pkg.API ~/.julia/juliaup/julia-1.10.5+0.x64.linux.gnu/share/julia/stdlib/v1.10/Pkg/src/API.jl:1848
[ Info: Precompiling PeriodicOrbits [41be5fce-5647-450b-ae37-a6739b881a1c]


Next, we give an initial guess of the location of the periodic orbit and its period.

In [ ]:
u0_guess = SVector(3.5, 3.0, 0.0)
T_guess = 5.2
ig = InitialGuess(u0_guess, T_guess) 

Then, we pick an appropriate algorithm that will detect the periodic orbit. In this case, we can use any algorithm intended for continuous-time dynamical systems. We choose the Optimized Shooting algorithm.

In [ ]:
alg = OptimizedShooting(Δt = 0.01, n = 3)

Finally, we combine all the pieces to find the periodic orbit.

In [ ]:
po = periodic_orbit(ds, alg, ig)
po

The closed curve of the periodic orbit can be visualized using a plotting library such as Makie.

In [ ]:
using CairoMakie

u0 = po.points[1]
T = po.T
traj, t = trajectory(ds, T, u0; Dt = 0.01)

fig = Figure()
ax = Axis3(fig[1, 1], azimuth = 0.6pi, elevation = 0.1pi)
lines!(ax, traj[:, 1], traj[:, 2], traj[:, 3], color = :blue, linewidth = 1.7)
scatter!(ax, u0)
fig

To ensure that the detected period is minimal, i.e., it is not a multiple of the minimal period, we can use the minimal_period function.

In [ ]:
minT_po = minimal_period(ds, po)
minT_po

To check whether two periodic orbits are equivalent up to some tolerance, use the poequal function.

In [ ]:
equal = poequal(po, minT_po; dthres = 1e-3, Tthres = 1e-3)
"Detected periodic orbit had minimal period: $(equal)"

To determine whether the found periodic orbit is stable or unstable, we can apply the isstable function.

In [ ]:
po = isstable(ds, po)
"Detected periodic orbit is $(po.stable ? "stable" : "unstable")."

# Acknowledgements

I would like to thank my mentor George Datseris who has been incredibly supportive over the course of the program. Furthermore, I want to thank to The Julia Language organization and GSOC program team who made this opportunity possible.

# Conclusion

This project successfully achieved its initial goal of creating a unified interface for periodic orbit detection algorithms. While there is still work to be done in terms of integrating more libraries and adding new algorithms, the foundation is solid, and the package is ready for further development and community contributions.